In this blog, we discuss how to solve semidefinite programs (SDPs) in ``Julia`` using ``Convex.jl``. We consider optimization problem of the form: 
$$
\begin{align*}
\begin{array}{ll}
\textrm{minimize} & \mathbf{trace}(CX)\\
\textrm{subject to} & \mathbf{trace}(A_{i}X)=b_{i},\\
 & X\succeq0,
\end{array} & i=1,\ldots,m
\end{align*}
$$
 where $X\in\mathbf{S}^{n}$ is the decision variable, and each of the $A_{i}$ matrices and $C$ are also in $\mathbf{S}^{n}$. By the notation $\mathbf{S}^{n}$, we denote the set of all symmetric $n\times n$ matrices.  

In [1]:
using SCS, Convex, COSMO, MosekTools, JuMP

using BenchmarkTools

In [2]:
# Create random data, change it to your problem

In [3]:
function random_mat_create(n)
    # this function creates a symmetric n×n matrix
    A = randn(n,n)
    A = A'*A
    A = (A+A')/2
    return A
end

random_mat_create (generic function with 1 method)

In [4]:
n = 10
m = 20
# set of all data matrices A_i
# the data matrix A = [A1 A2 A3 ....]
A = zeros(n, m*n) 
b = zeros(m)
# just ensuring our problem is feasible
X_test = rand(n,n)
X_test = X_test'*X_test
X_test = (X_test+X_test')/2
for i in 1:m
    A[:, (i-1)*n+1:i*n] .= random_mat_create(n)
    b[i] = tr(A[:, (i-1)*n+1:i*n]*X_test)
end
C = abs.(random_mat_create(n))

10×10 Array{Float64,2}:
 11.3257    3.03593     7.57764    …  1.36491    4.65387   3.49523
  3.03593   7.20755     0.0377997     0.841967   2.47635   3.17334
  7.57764   0.0377997  17.4409        0.0187263  4.68217   2.55754
  5.40527   3.38643     4.52541       3.12516    0.533301  0.642536
  0.442822  0.997689    0.324633      0.524024   0.990767  2.26337
  1.09737   1.40945     7.66599    …  5.17337    3.36144   0.807311
  3.33295   5.67138     6.1781        2.31512    5.3131    1.16169
  1.36491   0.841967    0.0187263     5.29625    0.137726  3.41034
  4.65387   2.47635     4.68217       0.137726   6.94162   5.4132
  3.49523   3.17334     2.55754       3.41034    5.4132    8.97888

In [5]:
A

10×200 Array{Float64,2}:
  13.1009      6.70666   0.0719756  …  -2.22339   -1.67688    3.32907
   6.70666    14.9119    1.43749        0.116857  -5.03077   -1.08398
   0.0719756   1.43749   3.78868        4.61666    3.70874   -2.8161
   0.745811    2.58424  -0.469766      -3.98044   -0.019262  -2.95362
  -1.99437    -2.82675  -0.535965       4.11535   -4.32979   -0.69598
  -1.49288    -2.01312   0.820385   …  -1.41683   -0.310927  -0.00789113
   6.04074     3.9018    1.05097        1.42818   -0.58277    1.15174
 -11.9482     -1.63738  -2.54123        9.20298    0.934847  -0.794667
   3.05548     1.26269   4.21032        0.934847   7.06475    0.675791
   3.94524     4.11502   0.235034      -0.794667   0.675791   4.82439

In [6]:
X_test

10×10 Array{Float64,2}:
 3.38728  1.64042  2.91841  3.08396  …  2.53948  3.39211  1.89857  2.78909
 1.64042  1.6409   2.41176  2.1642      2.05764  1.97808  1.25507  2.01304
 2.91841  2.41176  4.25385  3.65073     3.73043  3.53905  2.60916  3.61432
 3.08396  2.1642   3.65073  3.78324     3.58153  3.60734  2.19029  3.39561
 2.93006  1.71048  3.34021  3.15069     3.06394  3.44829  2.62906  3.18696
 2.10411  1.50142  2.8656   2.24801  …  2.9016   3.17009  1.98054  1.88816
 2.53948  2.05764  3.73043  3.58153     4.14588  3.54763  2.24665  3.004
 3.39211  1.97808  3.53905  3.60734     3.54763  4.15601  2.35485  3.09138
 1.89857  1.25507  2.60916  2.19029     2.24665  2.35485  2.12163  2.50346
 2.78909  2.01304  3.61432  3.39561     3.004    3.09138  2.50346  4.0686

In [9]:

function solve_SDP(A, b, C; solver_name=:COSMO)

# Create variable
    if solver_name == :COSMO
        model = Model(with_optimizer(COSMO.Optimizer))
    elseif solver_name == :Mosek
        model = Model(optimizer_with_attributes(Mosek.Optimizer))
    end

    set_silent(model)

    @variable(model, X[1:n, 1:n], PSD)


    @objective(model, Min, tr(C * X));
    for j in 1:m
        A_j = A[:, (j - 1) * n + 1:j * n]
        @constraint(model, tr(A_j * X) == b[j])
    end

    optimize!(model)

    status = JuMP.termination_status(model)
    X_sol = JuMP.value.(X)
    obj_value = JuMP.objective_value(model)

    return status, X_sol, obj_value

end

solve_SDP (generic function with 1 method)

In [12]:
status, X_sol, obj_value = solve_SDP(A, b, C; solver_name=:Mosek)

(MathOptInterface.OPTIMAL, [2.907044311952373 1.7130367276575142 … -0.056145513617222656 3.0230926674218024; 1.7130367276575142 3.419039624378557 … 1.0871948703965775 2.0577919984154334; … ; -0.056145513617222656 1.0871948703965775 … 0.7127834057861842 0.5195987956934747; 3.0230926674218024 2.0577919984154334 … 0.5195987956934747 4.234108180247669], 939.2696385581793)

Lets see which solver is faster, COSMO or Mosek.

In [10]:
@benchmark solve_SDP(A, b, C; solver_name=:COSMO)

BenchmarkTools.Trial: 
  memory estimate:  4.33 MiB
  allocs estimate:  35786
  --------------
  minimum time:     36.964 ms (0.00% GC)
  median time:      40.552 ms (0.00% GC)
  mean time:        40.787 ms (1.14% GC)
  maximum time:     46.920 ms (9.97% GC)
  --------------
  samples:          123
  evals/sample:     1

In [11]:
@benchmark solve_SDP(A, b, C; solver_name=:Mosek)

BenchmarkTools.Trial: 
  memory estimate:  3.81 MiB
  allocs estimate:  32015
  --------------
  minimum time:     6.647 ms (0.00% GC)
  median time:      7.524 ms (0.00% GC)
  mean time:        8.374 ms (5.00% GC)
  maximum time:     19.410 ms (25.35% GC)
  --------------
  samples:          597
  evals/sample:     1

So, on average Mosek seems to be 5 times faster than COSMO. 